## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-25-01-05-30 +0000,nyt,This Is the End of China’s Once Mightiest Prop...,https://www.nytimes.com/2025/08/24/business/ch...
1,2025-08-25-01-03-35 +0000,bbc,Chinese property giant Evergrande delisted aft...,https://www.bbc.com/news/articles/c14g7r44566o...
2,2025-08-25-01-02-52 +0000,48hills,How SF Democratic Party has become the right w...,https://48hills.org/2025/08/how-sf-democratic-...
3,2025-08-25-01-00-00 +0000,wsj,Tech Rally Shows Signs of Losing Steam,https://www.wsj.com/finance/stocks/tech-rally-...
4,2025-08-25-01-00-00 +0000,wsj,Inside Intel’s Tricky Dance With Trump,https://www.wsj.com/tech/inside-intels-tricky-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2440/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,24
444,ukraine,13
331,new,12
183,russia,10
290,war,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
85,2025-08-24-18-08-34 +0000,nypost,Russia’s top diplomat says Zelensky can’t sign...,https://nypost.com/2025/08/24/world-news/russi...,62
105,2025-08-24-14-39-41 +0000,nypost,Trump announces he’ll attend Ryder Cup in New ...,https://nypost.com/2025/08/24/us-news/trump-an...,60
84,2025-08-24-18-08-34 +0000,nypost,Vance reveals what Putin has conceded in Ukrai...,https://nypost.com/2025/08/24/us-news/vance-re...,56
70,2025-08-24-19-42-25 +0000,nyt,Trump Says Chicago and New York Are Next for F...,https://www.nytimes.com/2025/08/22/us/politics...,54
116,2025-08-24-14-05-19 +0000,nypost,Russia’s foreign minister throws up a new cond...,https://nypost.com/2025/08/24/us-news/russias-...,54


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
85,62,2025-08-24-18-08-34 +0000,nypost,Russia’s top diplomat says Zelensky can’t sign...,https://nypost.com/2025/08/24/world-news/russi...
70,54,2025-08-24-19-42-25 +0000,nyt,Trump Says Chicago and New York Are Next for F...,https://www.nytimes.com/2025/08/22/us/politics...
81,37,2025-08-24-18-30-00 +0000,nypost,Australian mom accused of killing American ste...,https://nypost.com/2025/08/24/world-news/lisa-...
125,32,2025-08-24-12-00-00 +0000,wsj,Here’s what you need to know about the unusual...,https://www.wsj.com/politics/policy/texas-cali...
56,29,2025-08-24-20-46-12 +0000,nypost,‘Sex and the City’ fans honor ‘iconic’ star Ki...,https://nypost.com/2025/08/24/us-news/sex-and-...
116,27,2025-08-24-14-05-19 +0000,nypost,Russia’s foreign minister throws up a new cond...,https://nypost.com/2025/08/24/us-news/russias-...
65,26,2025-08-24-20-14-54 +0000,nypost,Florida sheriff shows off $50K chain snatched ...,https://nypost.com/2025/08/24/us-news/florida-...
166,26,2025-08-24-02-00-00 +0000,wsj,Bashing the president’s tariff policies could ...,https://www.wsj.com/politics/policy/gop-tariff...
136,26,2025-08-24-10-24-22 +0000,nypost,Russia accuses Kyiv of striking nuclear power ...,https://nypost.com/2025/08/24/world-news/russi...
155,22,2025-08-24-05-48-50 +0000,nypost,Rescue effort halted for stuck climber on trea...,https://nypost.com/2025/08/24/world-news/kyrgy...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
